In [ ]:
import gym
import matplotlib
import numpy as np
import sys

from collections import defaultdict
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.blackjack import BlackjackEnv
from lib import plotting

matplotlib.style.use('ggplot')

In [ ]:
env = BlackjackEnv(natural=True)

In [ ]:
def make_epsilon_greedy_policy(Q, epsilon, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function and epsilon.
    
    Args:
        Q: A dictionary that maps from state -> action-values.
            Each value is a numpy array of length nA (see below)
        epsilon: The probability to select a random action . float between 0 and 1.
        nA: Number of actions in the environment.
    
    Returns:
        A function that takes the observation as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.
    
    """
    def policy_fn(observation):
        A = np.ones(nA, dtype=float) * epsilon / nA
        best_action = np.argmax(Q[observation])
        A[best_action] += (1.0 - epsilon)
        action = np.random.choice(np.arange(len(A)), p=A)
        return action
    return policy_fn

In [ ]:
def mc_control_epsilon_greedy(env, num_episodes, discount_factor=1.0, epsilon=0.1):
    """
    Monte Carlo Control using Epsilon-Greedy policies.
    Finds an optimal epsilon-greedy policy.
    
    Args:
        env: OpenAI gym environment.
        num_episodes: Number of episodes to sample.
        discount_factor: Gamma discount factor.
        epsilon: Chance the sample a random action. Float betwen 0 and 1.
    
    Returns:
        A tuple (Q, policy).
        Q is a dictionary mapping state -> action values.
        policy is a function that takes an observation as an argument and returns
        action probabilities
    """

    # Keeps track of sum and count of returns for each state
    # to calculate an average. We could use an array to save all
    # returns (like in the book) but that's memory inefficient.
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)

    # The final action-value function.
    # A nested dictionary that maps state -> (action -> action-value).
    Q = defaultdict(lambda: np.zeros(env.action_space.n))

    # The policy we're following
    policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)

    for episodes in range(1, num_episodes + 1):
        # Print out which episode we're on, useful for debugging.
        if episodes % 1000 == 0:
            print("\rEpisode {}/{}.".format(episodes, num_episodes), end="")
            sys.stdout.flush()
        
        episodes_s_a = []
        observation = env.reset()
        for t in range(100):
            action = policy(observation)
            new_observation, reward, is_done, _ = env.step(action)
            episodes_s_a.append([observation, action, reward])
            if is_done:
                break
            observation = new_observation
        for state, action in set((x[0], x[1]) for x in episodes_s_a):
            first_visit_s_a = next(i for i, x in enumerate(episodes_s_a)
                                   if x[0] == state and x[1] == action)
            g = sum(x[2] * discount_factor ** i for i, x in
                    enumerate(episodes_s_a[first_visit_s_a:]))
            returns_count[state, action] += 1.0
            returns_sum[state, action] += g
            Q[state][action] = returns_sum[state, action] / returns_count[state, action]
    # Implement this!

    return Q, policy


In [ ]:
Q, policy = mc_control_epsilon_greedy(env, num_episodes=500000, epsilon=0.1)

In [ ]:
# For plotting: Create value function from action-value function
# by picking the best action at each state
V = defaultdict(float)
for state, actions in Q.items():
    action_value = np.max(actions)
    V[state] = action_value
plotting.plot_value_function(V, title="Optimal Value Function")